### Bronze to Silver

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, DoubleType, StructType, FloatType

catalog_name = 'ecommerce'

### Brands

In [0]:
df_bronze = spark.table(f'{catalog_name}.bronze.brz_brands')
df_bronze.show(10)

In [0]:
df_silver = df_bronze.withColumn("brand_name", F.trim(F.col("brand_name")))
df_silver.show(10)

In [0]:
df_silver = df_bronze.withColumn("brand_code", F.regexp_replace(F.col("brand_code"), r'[A-Za-z0-9]', ''))
df_silver.show(10)

In [0]:
df_silver.select("category_code").distinct().show()

In [0]:
#Anomalies
anomalies = {
    "GROCERY": "GRCY",
    "BOOKS": "BKS",
    "TOYS": "TOY"
}

df_silver = df_silver.replace(to_replace=anomalies, subset=["category_code"])

df_silver.select("category_code").distinct().show()

In [0]:
#Save the corrected brands as slv_brands under silver layer

df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_brands")

### Category

In [0]:
df_bronze = spark.table(f'{catalog_name}.bronze.brz_category')

df_bronze.show(10)

In [0]:
#Removing Duplicates

df_duplicates = df_bronze.groupBy("category_code").count().filter(F.col("count") > 1)

display(df_duplicates)

In [0]:
df_silver = df_bronze.dropDuplicates(["category_code"])

display(df_silver)

In [0]:
df_silver = df_silver.withColumn("category_code", F.upper(F.col("category_code")))
display(df_silver)

In [0]:
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_category")

###Products

In [0]:
df_bronze = spark.read.table(f"{catalog_name}.bronze.brz_products")

row_count, column_count = df_bronze.count(), len(df_bronze.columns)

print(f"Row count: {row_count}")
print(f"Column count: {column_count}")

In [0]:
display(df_bronze.limit(5))

In [0]:
#Check and replace the 'g' from weight_grams
df_bronze.select("weight_grams").show(5, truncate=False)

#replace 'g' with empty string
df_silver = df_bronze.withColumn(
    "weight_grams",
    F.regexp_replace(F.col("weight_grams"), "g", "").cast(IntegerType())
)

#show 
df_silver.select("weight_grams").show(5, truncate=False)

In [0]:
#Check and replace the ',' from lenngth_cm
df_bronze.select("length_cm").show(5, truncate=False)

#replace 'g' with empty string
df_silver = df_bronze.withColumn(
    "length_cm",
    F.regexp_replace(F.col("length_cm"), ",", ".").cast(FloatType())
)

#show 
df_silver.select("length_cm").show(5, truncate=False)

In [0]:
#Category_code and brand_code to upperCase
df_silver.select("category_code", "brand_code").show(5)